In [1]:
import pandas as pd
import json
import numpy as np
import csv
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

C:\Users\joaog\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
nba = pd.read_json("NBAdataJSON.json")

In [3]:
nba_df = pd.DataFrame(nba)
nba_df

,player_id,birthDate,draft_year,name,position,college,salary,season_end,season_start,team,lat,long
4,acyqu01,"October 6, 1990",2012,Quincy Acy,Power Forward and Small Forward,Baylor University,1914544,2017,2016,Brooklyn Nets,31.546913,-97.121100
8,jackspi01,"August 29, 1991",2013,Pierre Jackson,Point Guard,Baylor University,750000,2016,2015,Philadelphia 76ers,31.546913,-97.121100
10,jeffeco01,"December 26, 1990",2014,Cory Jefferson,Power Forward,Baylor University,507336,2015,2014,Brooklyn Nets,31.546913,-97.121100
15,jonespe01,"September 24, 1991",2012,Perry Jones,Small Forward and Power Forward,Baylor University,2038206,2016,2015,Boston Celtics,31.546913,-97.121100
17,millequ01,"November 18, 1992",2012,Quincy Miller,Power Forward,Baylor University,788872,2014,2013,Denver Nuggets,31.546913,-97.121100
...,...,...,...,...,...,...,...,...,...,...,...,...
36804,williel01,"June 20, 1989",2010,Elliot Williams,Point Guard and Shooting Guard,"Duke University, University of Memphis",1442880,2013,2012,Portland Trail Blazers,35.118750,-89.937493
36813,williel01,"June 20, 1989",2010,Elliot Williams,Point Guard and Shooting Guard,"Duke University, University of Memphis",1442880,2013,2012,Portland Trail Blazers,35.118750,-89.937493
36822,youngjo01,"June 27, 1992",2015,Joe Young,Point Guard,"University of Houston, University of Oregon",1471382,2018,2017,Indiana Pacers,44.044830,-123.072605
36825,youngjo01,"June 27, 1992",2015,Joe Young,Point Guard,"University of Houston, University of Oregon",1471382,2018,2017,Indiana Pacers,44.044830,-123.072605


In [30]:
nba_df_small = nba_df.head(10)

In [5]:
list_positions = ["Power Forward", "Point Guard", "Small Forward", "Shooting Guard", "Center"]

In [6]:
list_positions[0][3]

'e'

In [17]:
def match_position(player, draft_position, list_positions, min_score=0):
    # 0 score incase we don't get any matches
     
    max_score = 0

    # Returning empty name for no match as well
    max_position = ""
#     max_name2 = ""

    # Iternating over all names (transfermartk df) in two list of names (shor and long in fifa_20 df)
    if len(draft_position) > 14:
        draft_position = draft_position[0:14]

    for position in list_positions:
        score = fuzz.ratio(position, draft_position )
        # Checking if we are above our threshold and have a better score
        if (score > min_score) & (score > max_score):
            max_position = position
            max_score = score

#     
    # return the names matched for the two lists
    return (max_position, max_score, draft_position)

In [18]:
dict_list = []

# iterating over players in transfermarkt
for player in nba_df_small.player_id:
    # Use our method to find best match, we can set a threshold here 
    match = match_position(player, nba_df_small.position, list_positions, 70)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({"player_id" : player})
    dict_.update({"draft_position" : match[2]})
    dict_.update({"position" : match[0]})
    dict_.update({"position_score" : match[1]})

    dict_list.append(dict_)
    
merge_table_names = pd.DataFrame(dict_list)

# Display results
merge_table_names

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [21]:
def slicer(player, df_position):

    for draft_position in df_position:
        if len(draft_position) > 14:
            draf_position = draft_position[0:14]
            
    return (draft_position)

In [23]:
dict_list1 = []

# iterating over players in transfermarkt
for player in nba_df_small.player_id:
    # Use our method to find best match, we can set a threshold here 
    sliced = slicer(player, nba_df_small.position)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({"player_id" : player})
    dict_.update({"sliced_position" : sliced[0]})

    dict_list.append(dict_)
    
sliced_table = pd.DataFrame(dict_list1)

# Display results
sliced_table

""


In [56]:
# iterating over players in transfermarkt
new_position = []
for draft_position in nba_df_small.position:
    position = draft_position.split(" ")
    if len(position) > 1:
        position_clear = position[0] + " " +position[1]
    else:
        position_clear = position[0]
    new_position.append(position_clear)
    
print(new_position)

['Power Forward', 'Point Guard', 'Power Forward', 'Small Forward', 'Power Forward', 'Small Forward', 'Power Forward', 'Point Guard', 'Power Forward', 'Small Forward']


In [54]:
nba_df_small["position"] = new_position

C:\Users\joaog\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [55]:
nba_df_small

,player_id,birthDate,draft_year,name,position,college,salary,season_end,season_start,team,lat,long
4,acyqu01,"October 6, 1990",2012,Quincy Acy,Power Forward,Baylor University,1914544,2017,2016,Brooklyn Nets,31.546913,-97.1211
8,jackspi01,"August 29, 1991",2013,Pierre Jackson,Point Guard,Baylor University,750000,2016,2015,Philadelphia 76ers,31.546913,-97.1211
10,jeffeco01,"December 26, 1990",2014,Cory Jefferson,Power Forward,Baylor University,507336,2015,2014,Brooklyn Nets,31.546913,-97.1211
15,jonespe01,"September 24, 1991",2012,Perry Jones,Small Forward,Baylor University,2038206,2016,2015,Boston Celtics,31.546913,-97.1211
17,millequ01,"November 18, 1992",2012,Quincy Miller,Power Forward,Baylor University,788872,2014,2013,Denver Nuggets,31.546913,-97.1211
23,princta02,"March 22, 1994",2016,Taurean Prince,Small Forward,Baylor University,2422560,2018,2017,Atlanta Hawks,31.546913,-97.1211
28,acyqu01,"October 6, 1990",2012,Quincy Acy,Power Forward,Baylor University,1914544,2017,2016,Brooklyn Nets,31.546913,-97.1211
32,jackspi01,"August 29, 1991",2013,Pierre Jackson,Point Guard,Baylor University,750000,2016,2015,Philadelphia 76ers,31.546913,-97.1211
34,jeffeco01,"December 26, 1990",2014,Cory Jefferson,Power Forward,Baylor University,507336,2015,2014,Brooklyn Nets,31.546913,-97.1211
39,jonespe01,"September 24, 1991",2012,Perry Jones,Small Forward,Baylor University,2038206,2016,2015,Boston Celtics,31.546913,-97.1211
